In [2]:
import labelbox
import json
import os
import urllib
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from urllib.parse import urlparse, unquote
import matplotlib.pyplot as plt
import password
import requests
import pickle

In [22]:
class FetchImages:
    def __init__(self, lb_api_key, project_id):
        self.lb_api_key = lb_api_key
        self.project_id = project_id

    def preprocess_data(self, data, size=(640, 640), save_to_folder=True):
        images = []
        labels = []
        annotations = []

        if save_to_folder:
            if not os.path.exists("temp_images"):
                os.makedirs("temp_images")
            if not os.path.exists("temp_data"):
                os.makedirs("temp_data")

        for entry in data:
            try:
                img_url = entry["Labeled Data"]
                response = requests.get(img_url, stream=True)
                response.raise_for_status()

                img_bytes = response.content
                img_array = np.frombuffer(img_bytes, dtype=np.uint8)
                img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

                if img is None:
                    print(f"Nie można wczytać obrazu: {img_url}")
                    continue

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                bbox = entry["Label"]["objects"][0]["bbox"]
                if bbox["height"] <= 0 or bbox["width"] <= 0:
                    print(f"Nieprawidłowe współrzędne obrazu: {img_url}")
                    continue

                plate = img[
                    bbox["top"] : bbox["top"] + bbox["height"],
                    bbox["left"] : bbox["left"] + bbox["width"],
                ]

                if plate.size == 0:
                    print(f"Nie można przeskalować obrazu: {img_url}")
                    continue

                # Scaling the image
                scale_ratio_y = size[0] / img.shape[0]
                scale_ratio_x = size[1] / img.shape[1]
                img_resized = cv2.resize(img, size)

                # Scaling the bounding box coordinates
                bbox_scaled = {
                    "top": int(bbox["top"] * scale_ratio_y),
                    "left": int(bbox["left"] * scale_ratio_x),
                    "height": int(bbox["height"] * scale_ratio_y),
                    "width": int(bbox["width"] * scale_ratio_x),
                }

                if save_to_folder:
                    img_name = entry["External ID"]
                    img_path = os.path.join("temp_images", img_name)
                    cv2.imwrite(img_path, cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR))

                images.append(img_resized)
                labels.append(entry["External ID"])
                annotations.append(bbox_scaled)

            except Exception as e:
                print(f"Wystąpił błąd podczas przetwarzania obrazu: {img_url}")
                print(f"Szczegóły błędu: {str(e)}")

        if save_to_folder:
            with open("temp_data/images.pkl", "wb") as f:
                pickle.dump(images, f)
            with open("temp_data/annotations.pkl", "wb") as f:
                pickle.dump(annotations, f)
            with open("temp_data/labels.pkl", "wb") as f:
                pickle.dump(labels, f)

    def fetch_data(self):
        # Labelbox API key here
        lb = labelbox.Client(api_key=self.lb_api_key)

        # Get project by ID
        project = lb.get_project(self.project_id)

        # Export labels created in the selected date range as a json file:
        labels = project.export_labels(
            download=True, start="2023-04-02", end="2023-05-02"
        )

        self.preprocess_data(labels)

    def load_data(self):
        if (
            os.path.exists("temp_data/images.pkl")
            and os.path.exists("temp_data/annotations.pkl")
            and os.path.exists("temp_data/labels.pkl")
        ):
            with open("temp_data/images.pkl", "rb") as f:
                images = pickle.load(f)
            with open("temp_data/annotations.pkl", "rb") as f:
                annotations = pickle.load(f)
            with open("temp_data/labels.pkl", "rb") as f:
                labels = pickle.load(f)

            return np.array(images), np.array(annotations), np.array(labels)

    def create_annotations_structure(self, images, annotations, labels):
        annotations_structure = []

        for image, annotation, label in zip(images, annotations, labels):
            height, width, _ = image.shape
            objects = []

            obj = {
                "class": "license",
                "x_min": annotation["left"],
                "y_min": annotation["top"],
                "x_max": annotation["left"] + annotation["width"],
                "y_max": annotation["top"] + annotation["height"],
            }
            objects.append(obj)

            ann_structure = {
                "filename": label,
                "width": width,
                "height": height,
                "objects": objects,
            }
            annotations_structure.append(ann_structure)

        return annotations_structure

    def convert_to_yolo_format(self, annotations, output_dir):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        class_mapping = {
            "license": 0
        }  # mapowanie klasy na indeks (w tym przypadku mamy tylko jedną klasę)

        for annotation in annotations:
            img_path = annotation["filename"]
            img_width = annotation["width"]
            img_height = annotation["height"]
            objects = annotation["objects"]

            yolo_labels = []
            for obj in objects:
                class_name = obj["class"]
                class_index = class_mapping[class_name]

                x_min, y_min, x_max, y_max = (
                    obj["x_min"],
                    obj["y_min"],
                    obj["x_max"],
                    obj["y_max"],
                )

                # Obliczanie współrzędnych środkowego punktu, szerokości i wysokości Bounding Box
                x_center = (x_min + x_max) / 2
                y_center = (y_min + y_max) / 2
                width = x_max - x_min
                height = y_max - y_min

                # Normalizacja wartości do zakresu 0-1
                x_center /= img_width
                y_center /= img_height
                width /= img_width
                height /= img_height

                yolo_label = f"{class_index} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                yolo_labels.append(yolo_label)

            # Zapis etykiet do pliku .txt
            base_filename = os.path.splitext(os.path.basename(img_path))[0]
            output_path = os.path.join(output_dir, f"{base_filename}.txt")
            with open(output_path, "w") as f:
                f.write("\n".join(yolo_labels))

In [23]:
# Labelbox API key here
LB_API_KEY = password.LB_API_KEY

PROJECT_ID = "clh13dzrw0l4a07zmf36u97rd"

image = FetchImages(LB_API_KEY, PROJECT_ID)

In [ ]:
image.fetch_data()

In [24]:
images, annotations, labels = image.load_data()

In [25]:
annotations_structure = image.create_annotations_structure(images, annotations, labels)

In [26]:
image.convert_to_yolo_format(
    annotations=annotations_structure, output_dir="yolo_format_plate"
)

In [27]:
images_folder = "temp_images"

# Pobierz listę wszystkich plików w folderze
all_files = os.listdir(images_folder)

# Stwórz listę zawierającą tylko ścieżki do plików z rozszerzeniem .jpg
# image_paths = [os.path.join(images_folder, file) for file in all_files if file.endswith('.jpeg')]
image_paths = [file for file in all_files if file.endswith(".jpeg")]

In [28]:
train_images, val_images = train_test_split(image_paths, test_size=0.2, random_state=42)

In [29]:
with open("train.txt", "w") as f:
    for img_path in train_images:
        f.write(f"{img_path}\n")

with open("val.txt", "w") as f:
    for img_path in val_images:
        f.write(f"{img_path}\n")